In [1]:
import sys
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
sys.path.append('/Users/christinagosnell/code/pudl')
sys.path.append('/Users/Nannerz/Desktop/working/pudl/')
from pudl import ferc1, pudl, models, models_ferc1, settings, constants
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
%load_ext autoreload
%autoreload 1

In [2]:
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.models
%aimport pudl.models_ferc1
%aimport pudl.eia923
%aimport pudl.constants

In [3]:
ferc1_engine = pudl.ferc1.db_connect_ferc1()
pudl.ferc1.init_db(refyear=2015, years=range(2004,2016), def_db=True)

Defining new FERC Form 1 DB based on 2015...
Ingesting FERC Form 1 Data from 2004...
Ingesting FERC Form 1 Data from 2005...
Ingesting FERC Form 1 Data from 2006...
Ingesting FERC Form 1 Data from 2007...
Ingesting FERC Form 1 Data from 2008...
Ingesting FERC Form 1 Data from 2009...
Ingesting FERC Form 1 Data from 2010...
Ingesting FERC Form 1 Data from 2011...
Ingesting FERC Form 1 Data from 2012...
Ingesting FERC Form 1 Data from 2013...
Ingesting FERC Form 1 Data from 2014...
Ingesting FERC Form 1 Data from 2015...


In [4]:
ferc1_respondents = pd.read_sql('''SELECT respondent_id, respondent_name FROM f1_respondent_id''', ferc1_engine)
ferc1_steam_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, tot_capacity
                                    FROM f1_steam''', ferc1_engine)
ferc1_small_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, kind_of_fuel, capacity_rating
                                    FROM f1_gnrt_plant''', ferc1_engine)
ferc1_hydro_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, plant_kind, tot_capacity
                                    FROM f1_hydro''', ferc1_engine)
ferc1_pumped_storage_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, plant_kind, tot_capacity
                                    FROM f1_pumped_storage''', ferc1_engine)

In [5]:
ferc1_all_plants = pd.concat([ferc1_steam_plants, ferc1_small_plants, ferc1_hydro_plants, ferc1_pumped_storage_plants])
ferc1_all_plants = ferc1_respondents.merge(ferc1_all_plants, how='left', on='respondent_id')
ferc1_all_plants = ferc1_all_plants.replace('^$', np.nan, regex=True)
ferc1_all_plants = ferc1_all_plants.dropna(subset=['plant_name',])
ferc1_all_plants['report_year'] = ferc1_all_plants.report_year.astype(int)
ordered_for_cgosnell = ['respondent_id', 'respondent_name', 'plant_name', 'report_year',\
                        'kind_of_fuel',  'capacity_rating', 'plant_kind', 'tot_capacity']
ferc1_all_plants = ferc1_all_plants[ordered_for_cgosnell]
ferc1_all_plants = ferc1_all_plants.drop_duplicates(subset=['respondent_id','plant_name'])

In [6]:
ferc1_all_plants.to_csv('../../results/id_mapping/ferc1_all_plants_2004-2015.csv', index=False)

In [7]:
ferc1_respondents.to_csv('../../results/id_mapping/ferc1_all_respondents_2004-2015.csv', index=False)